In [1]:
import os
import json
import datetime
import csv
import nltk
import ssl

import streamlit as st
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# SSL context for nltk
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath("nltk_data"))
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
intents=[
    {
      "tag": "greeting",
      "patterns": ["Hello", "Hi", "Hey", "Good morning", "Good evening"],
      "responses": ["Hi there!", "Hello! How can I assist you?", "Greetings!"]
    },
    {
      "tag": "goodbye",
      "patterns": ["Bye", "Goodbye", "See you later", "Take care"],
      "responses": ["Goodbye! Have a great day!", "See you later!", "Take care!"]
    },
    {
      "tag": "thanks",
      "patterns": ["Thanks", "Thank you", "I appreciate it"],
      "responses": ["You're welcome!", "Glad I could help!", "Anytime!"]
    },
    {
      "tag": "linear_regression",
      "patterns": [
        "What is linear regression?",
        "Explain linear regression",
        "Use cases of linear regression"
      ],
      "responses": [
        "Linear Regression is a supervised learning algorithm used for predicting continuous values.",
        "Some use cases include house price prediction and sales forecasting.",
        "It models the relationship between dependent and independent variables."
      ]
    },
    {
      "tag": "logistic_regression",
      "patterns": [
        "What is logistic regression?",
        "Explain logistic regression",
        "Applications of logistic regression"
      ],
      "responses": [
        "Logistic Regression is a classification algorithm used to predict binary outcomes.",
        "Common use cases are email spam detection and diabetes prediction.",
        "It outputs probabilities that map to classes."
      ]
    },
    {
      "tag": "decision_trees",
      "patterns": [
        "What are decision trees?",
        "Explain decision trees",
        "Applications of decision trees"
      ],
      "responses": [
        "Decision Trees use a tree-like structure to make decisions based on feature values.",
        "Common use cases are credit risk assessment and customer segmentation.",
        "They are simple to interpret and implement."
      ]
    },
    {
      "tag": "svm",
      "patterns": [
        "What is Support Vector Machines?",
        "Explain SVM",
        "Applications of SVM"
      ],
      "responses": [
        "Support Vector Machines (SVM) is a classification algorithm that separates data points with a hyperplane.",
        "Applications include image classification and text categorization.",
        "It is effective for high-dimensional spaces."
      ]
    },
    {
      "tag": "neural_networks",
      "patterns": [
        "What are neural networks?",
        "Explain neural networks",
        "Applications of neural networks"
      ],
      "responses": [
        "Neural Networks are algorithms modeled after the human brain, used for complex pattern recognition.",
        "Applications include image recognition and natural language processing.",
        "They consist of layers of interconnected nodes."
      ]
    },
    {
      "tag": "unsupervised_learning",
      "patterns": [
        "What is unsupervised learning?",
        "Explain unsupervised learning",
        "Examples of unsupervised learning"
      ],
      "responses": [
        "Unsupervised Learning is a type of machine learning where models find patterns in data without labels.",
        "Examples include clustering algorithms like K-Means.",
        "It helps uncover hidden structures in data."
      ]
    },
    {
      "tag": "overfitting",
      "patterns": [
        "What is overfitting?",
        "Explain overfitting",
        "How to prevent overfitting"
      ],
      "responses": [
        "Overfitting occurs when a model learns noise in the training data, reducing its ability to generalize.",
        "You can prevent it using techniques like cross-validation and regularization.",
        "Simplifying the model can also help reduce overfitting."
      ]
    },
    {
      "tag": "tools",
      "patterns": [
        "What are some tools for machine learning?",
        "Popular machine learning tools",
        "Examples of ML tools"
      ],
      "responses": [
        "Popular ML tools include TensorFlow, PyTorch, scikit-learn, and Keras.",
        "Other tools are Pandas for data manipulation and Matplotlib for visualization.",
        "For large-scale data, Apache Spark MLlib is commonly used."
      ]
    },
    {
      "tag": "common_questions",
      "patterns": [
        "What is machine learning?",
        "Difference between supervised and unsupervised learning",
        "What is feature scaling?"
      ],
      "responses": [
        "Machine Learning enables systems to learn from data and improve without being explicitly programmed.",
        "Supervised learning uses labeled data, while unsupervised learning identifies patterns in unlabeled data.",
        "Feature scaling normalizes the range of features in data for better performance."
      ]
    }
  ]
  

In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1, 4))
clf = LogisticRegression(random_state=0, max_iter=10000)

# Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

# Training the model
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

LogisticRegression(max_iter=10000, random_state=0)

In [6]:
def chatbot(input_text):
    input_text_transformed = vectorizer.transform([input_text])
    predicted_tag = clf.predict(input_text_transformed)[0]
    for intent in intents:
        if intent['tag'] == predicted_tag:
            return random.choice(intent['responses'])
    return "Sorry, I didn't understand that. Can you rephrase?"  # Default response

counter = 0

In [7]:
def main():
    global counter
    st.title("ML Chatbot")

    menu = ["Home", "Conversation History", "About"]
    choice = st.sidebar.selectbox("Menu", menu)

    if choice == "Home":
        st.write("Welcome to the ML Chatbot! Ask me anything about Machine Learning.")
        if not os.path.exists('chat_log.csv'):
            with open('chat_log.csv', 'w', newline='', encoding='utf-8') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerow(['User Input', 'Chatbot Response', 'Timestamp'])

        counter += 1
        user_input = st.text_input("You:", key=f"user_input_{counter}")

        if user_input:
            user_input_str = str(user_input)
            response = chatbot(user_input)

            st.text_area("Chatbot:", value=response, height=120, max_chars=None, key=f"chatbot_response_{counter}")

            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            with open('chat_log.csv', 'a', newline='', encoding='utf-8') as csvfile:
                csv_writer = csv.writer(csvfile)
                csv_writer.writerow([user_input_str, response, timestamp])

            if response.lower() in ['goodbye', 'bye']:
                st.write("Thank you for chatting with me about ML. Have a great day!")
                st.stop()

    elif choice == "Conversation History":
        st.header("Conversation History")
        with open('chat_log.csv', 'r', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)
            next(csv_reader)  # Skip header row
            for row in csv_reader:
                st.text(f"User: {row[0]}")
                st.text(f"Chatbot: {row[1]}")
                st.text(f"Timestamp: {row[2]}")
                st.markdown("---")

    elif choice == "About":
        st.write("This chatbot is designed to answer questions about Machine Learning using NLP.")
        st.subheader("Features:")
        st.write("""
        - Uses a logistic regression model to classify user intents.
        - Includes pre-trained intents for ML topics.
        """)

if __name__ == '__main__':
    main()

2024-12-08 18:00:51.828 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 18:00:53.012 
  command:

    streamlit run D:\ana\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-12-08 18:00:53.025 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 18:00:53.033 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 18:00:53.033 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 18:00:53.049 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 18:00:53.049 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-08 18:00:53.057 Session state does not function when runn